In [2]:
# Web scraper
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup

# Create list for urls: 
url_lst = []

for i in range(1, 51):
    response = requests.get('https://www.finn.no/realestate/lettings/search.html?page='+str(i)+'&sort=PUBLISHED_DESC')
    soup = BeautifulSoup(response.text, 'html.parser')

    # For loop to extract all urls from soup:
    for url in soup.find_all('a', href=True):
        soup.select('h2')[1].select('a', href=True)[0]['href']
        
        # Only keep urls with 'lettings' in them:
        if 'www.finn.no/realestate/lettings/' in url['href']: 
            url_lst.append(url['href'])
        else:
            pass
        
    # Sleep for random time between 1 and 2 seconds:
    time.sleep(random.randint(1000,2000)/1000)




In [3]:
# Create empty dataframe with the columns 'address', 'rent', 'url', and 'area':
df = pd.DataFrame(columns=['address', 'rent', 'area', 'url'])

for url_i in range(len(url_lst)): 
    response = requests.get(url_lst[url_i])
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract unique p class from soup (this is the address):
    try:    
        adr = soup.find_all('p', attrs={'class': 'u-caption'})[0].text
    except:
        adr = 'None'
    
    # Extract rent from soup:
    try:
        rent = soup.find('span', attrs={'class': 'u-t3'}).text
    except:
        pass
    
    # Extract primary info from soup:
    try:
        area = soup.find_all('dd')[2].text
    except: 
        pass

    url = url_lst[url_i]

    # Insert adr, rent, area, and url into df:
    df = df.append({'address': adr, 'rent': rent, 'area': area, 'url': url}, ignore_index=True)

    # Sleep for random time between .5 and 1 second:
    time.sleep(random.randint(500,1000)/1000)



C:\Users\marku\AppData\Local\Temp\ipykernel_11924\54485250.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'address': adr, 'rent': rent, 'area': area, 'url': url}, ignore_index=True)
C:\Users\marku\AppData\Local\Temp\ipykernel_11924\54485250.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'address': adr, 'rent': rent, 'area': area, 'url': url}, ignore_index=True)
C:\Users\marku\AppData\Local\Temp\ipykernel_11924\54485250.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'address': adr, 'rent': rent, 'area': area, 'url': url}, ignore_index=True)
C:\Users\marku\AppData\Local\Temp\ipykernel_11924\54485250.py:29: FutureWarning: The frame.append method is deprecated an

In [5]:
# Clean data: 
# In column address, replace comma with space: 
df['address'] = df['address'].str.replace(',', ' ')
df

# Create a column with the postal code from the address, which is 4 numbers long: 
df['postal_code'] = df['address'].str.extract('(\d{4})')

# Remove the 4 digit postal code from address column: 
df['address'] = df['address'].str.replace('\d{4}', '')

# Create a column with the city from the address:
df['city'] = df['address'].str.split(' ').str[-2] + ' ' + df['address'].str.split(' ').str[-1]

city_lst = df['city'].unique().tolist()

# Remove the column 'city' from the address column:
for city in city_lst:
    df['address'] = df['address'].str.replace(city, '')

# Clean 'area' column and only keep the number:
df['area'] = df['area'].str.replace('m²', '')

# Delete rows in 'area' that dont contain numbers:
df = df[df['area'].str.contains('\d')]
df['area'] = df['area'].str.replace('\.', '')

# Convert 'area' to integer: 
df['area'] = df['area'].astype(int)

# Remove 'area' that are less than 10:
if df['area'].min() < 10:
    df = df[df['area'] > 10]

# Remove ' ' and 'kr' from 'rent' column:
df['rent'] = df['rent'].str.replace(' kr', '')
df['rent'] = df['rent'].str.replace(' ', '')

# Add column rent per square meter:
df['rent_per_sqm'] = (df['rent'].astype(int) / df['area']).astype(int)

# Save df as csv with today's date:
df.to_csv('rent_data_norway_' + str(time.strftime("%Y%m%d")) + '.csv', index=False)

# To do: 
# 1. Add date column
# 2. Add column with number of rooms
# 3. Add column with number of bathrooms

# Overall goal: 
# Calculate estimated gross yield for each property 


C:\Users\marku\AppData\Local\Temp\ipykernel_11924\914067780.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address'] = df['address'].str.replace('\d{4}', '')


TypeError: first argument must be string or compiled pattern

In [3]:
import pandas as pd
test = pd.read_csv('rent_data_norway_20230113.csv')